In [1]:
# Самостоятельно найти сайт, соответствующий следующим условиям:
# непопулярный, регионального уровня или из узкой области (с целью избежать дублирования)
# наличие страниц-каталогов, где есть информация сразу по нескольких объектам
# наличие страниц, посвященных отдельному объекту
 
# Необходимо:
# спарсить нескольких страниц (минимум 10), посвященных только одному объекту;
# спарсить страницы-каталоги, где размещена информация сразу по нескольким объектам.
# Данные можно скачать и сохранить локально в виде html, 
# а можно организовать их получение напрямую через обращение к серверу сайта.
# Результаты парсинга собрать отдельно по каждой подзадаче и записать в отдельный json.
# Выполните произвольные операции с данными:
# отсортируйте значения по одному из доступных полей
# выполните фильтрацию по другому полю (запишите результат отдельно)
# для одного выбранного числового поля посчитайте показатели статистики
# для одного текстового поля посчитайте частоту меток.

In [12]:
# https://belleyou.ru

import requests as req
from bs4 import BeautifulSoup
import re
import json
import math
import collections
import pandas as pd
import numpy as np
import os

In [13]:
url = "https://belleyou.ru"

In [14]:
url_odejda = f"{url}/catalog/odezhda/"  # url_antib = f"{url_main}/catalog/antibiotiki/"
suburl_odejda_br = "bryuki/"  # suburl_antib_inj = "inekcionnye/"
suburl_odejda = [suburl_odejda_br]  # suburl_antib = [suburl_antib_inj] 

In [15]:
z_numb = 5
file_path = 'tests/' + str(z_numb)
filename = file_path + '/'

In [16]:
json_out_filename = f"{file_path}/"

In [17]:
bld_name_freq = {}
new_info = []
pl = []

In [18]:
odezhda_name_freq = {} # antib_name_freq = {}
odezhda_info = [] # antib_info = []
odezhda_pl = [] # antib_pl = []

In [19]:
odezhda_country_freq = {} # antib_country_freq = {}

In [20]:
# Определяем функцию, которая добавляет элемент в словарь.

def add_to_dict(m, fdict):
    m = m.lower()
    if m in fdict:
        fdict[m] += 1
    else:
        fdict.setdefault(m, 1)

In [23]:
# Определяем функцию, которая отправляет запрос по указанному URL-адресу и сохраняет ответ в виде HTML-файла. 
# Если код состояния не равен 200, вместо этого он пытается прочитать файл из локального каталога.

def get_html_file(url, purl):
    url_get = url + purl
    print("request to : ", url_get)
    resp = req.get(url_get)
    html_filename = f"dl/{purl.split('/')[-2]}.html"
    # print(html_filename)
    if resp.status_code != 200:
        with open(f"{file_path}/{html_filename}", mode="r") as html_f:
            resp = html_f.readlines()
        return "".join(resp)
    else:
        dl_dir = file_path + "/dl"
        if not os.path.exists(dl_dir):
            os.mkdir(dl_dir)
        if not os.path.exists(html_filename):
            with open(f"{file_path}/{html_filename}", mode = "w") as html_f:
                html_f.write(resp.text)
    return resp.text

In [ ]:
for curl in suburl_odejda:
    resp = get_html_file(url_odejda, curl)
    soup = BeautifulSoup(resp, 'html.parser')
    content = soup.find_all("div", {"class" : "catalog_item col-12 col-md-5 col-lg-3"})
    for cv in content:
        item = {}
        link = cv.find('a', {"class" : "art"})
        href = link.get('href')
        p_name = cv.find("p", {"class" : "name"})
        page = get_html_file(url_main, href)
        item['url'] = href
        item['name'] = p_name.text
        ps = BeautifulSoup(page, 'html.parser')
        discr = ps.find("div", {"class" : "col-12 col-md-7 product_page_description"})
        p_manuf = discr.find("p", {"class" : "manufacturer"})
        p_ishere = discr.find("p", {"class" : "is_here"})
        mname = " ".join(p_manuf.text.split()[1:])
        item['manufacturer'] = mname
        add_to_dict(mname, antib_name_freq)       
        count = 0
        if p_ishere:
            count = int(p_ishere.text.split()[1].replace(".", ""))
        item['count'] = count
        p_price = discr.find("span", {"class" : "price"})
        item['price'] = int(p_price.text.replace(" ", "")[0:-1])
        antib_pl.append(item['price'])
        tab_content = ps.find("div", {"class" : "tab-content"})
        t_features = tab_content.find("div", {"class" : "features"})
        span_li = iter(t_features.find_all("span"))
        item['features'] = {}
        item['features']['type'] = next(span_li).text
        item['features']['country'] = next(span_li).text
        add_to_dict(item['features']['country'], antib_country_freq)    
        item['features']['weight'] = next(span_li).text        
        f_count = 0
        try:
            f_count = int(next(span_li).text.replace(".", ""))
        except:
            pass            
        item['features']['count'] = f_count
        antib_info.append(item)